In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [6]:
!conda install -c conda-forge folium --yes

import folium # map rendering library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

    branca:  0.4.0-py_0  conda-forge
    folium:  0.10.1-py_0 conda-forge
    vincent: 0.4.4-py_1  conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 100% 
branca-0.4.0         | 26 KB     | ##################################### | 100% 
folium-0.10.1        | 59 KB   

In [7]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Get the html from the wikipedia page with the postal codes of Canada

In [8]:

soup = BeautifulSoup(data, 'html.parser')

In [27]:
postalCodeList = []
boroughList = []
neighborhoodList = []
latitude = []
longitude = []

In [28]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [30]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))
       

In [19]:
#Load the pandas dataframe
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [20]:
#Remove no-assigned columns
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [21]:
#Group neighbors in same borough
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
2,M1B\n,Scarborough\n,Malvern / Rouge
3,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
4,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek


In [22]:

for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
2,M1B\n,Scarborough\n,Malvern / Rouge
3,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
4,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek


###### Neighborhood WHICH IS "Not assigned", make the value the same as Borough

In [23]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street"
1,M2H,North York,"Hillcrest Village, Hillcrest Village"
2,M4B,East York,"Parkview Hill / Woodbine Gardens, Parkview Hil..."
3,M1J,Scarborough,"Scarborough Village, Scarborough Village"
4,M4G,East York,"Leaside, Leaside"
5,M4M,East Toronto,"Studio District, Studio District"
6,M1R,Scarborough,"Wexford / Maryvale, Wexford / Maryvale"
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,"Humber Summit, Humber Summit"
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


###### In the last cell used the .shape method to print the number of rows of the dataframe.

In [31]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(283, 3)

In [33]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,,NaN,NaN
1,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge",43.806686,-79.194353
2,M1B\n,Scarborough\n,Malvern / Rouge,NaN,NaN
3,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug...",43.784535,-79.160497
4,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek,NaN,NaN


In [36]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,"Central Bay Street, Central Bay Street",43.657952,-79.387383
1,M2H,North York,"Hillcrest Village, Hillcrest Village",43.803762,-79.363452
2,M4B,East York,"Parkview Hill / Woodbine Gardens, Parkview Hil...",43.706397,-79.309937
3,M1J,Scarborough,"Scarborough Village, Scarborough Village",43.744734,-79.239476
4,M4G,East York,"Leaside, Leaside",43.709060,-79.363452
5,M4M,East Toronto,"Studio District, Studio District",43.659526,-79.340923
6,M1R,Scarborough,"Wexford / Maryvale, Wexford / Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,"Humber Summit, Humber Summit",43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420
